In [1]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers, models
from keras.utils import to_categorical

from random import shuffle
from os import listdir


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
img_dir = "imgs/tweeked/"
file_list = listdir(img_dir)
labels = ([filename.split("_")[0] for filename in file_list])

classnames, labels = np.unique(labels, return_inverse=True)

images =([plt.imread(img_dir+filename)[:,:,:3] for filename in file_list])

In [3]:
image_shape = np.shape(images[1])
print(image_shape)

(600, 600, 3)


In [4]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

#config = tf.ConfigProto(
#        device_count = {'GPU': 0}
#    )
#sess = tf.Session(config=config)

In [5]:
idx = list(range(0,len(images)))

shuffle(idx)
divider = int(3*len(images)/4)
train_idx = [idx[i] for i in list(range(0, divider)) ]
test_idx = [idx[i] for i in list(range(divider+1, len(images)-1)) ]

images_train = np.asarray([images[i] for i in train_idx])
images_test = np.asarray([images[i] for i in test_idx])

labels_train = np.asarray([labels[i] for i in train_idx])
labels_test = np.asarray([labels[i] for i in test_idx])

In [6]:
model = models.Sequential()
model.add(layers.Conv2D(10, (5, 5), activation='relu', input_shape=image_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(10, (5, 5), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(10, (5, 5), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 596, 596, 10)      760       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 298, 298, 10)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 294, 294, 10)      2510      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 147, 147, 10)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 143, 143, 10)      2510      
_________________________________________________________________
flatten (Flatten)            (None, 204490)            0         
____

In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(images_train, labels_train, epochs=15, 
                    validation_data=(images_test, labels_test)),

Train on 504 samples, validate on 166 samples
Epoch 1/15
504/504 [==============================] - 64s 127ms/sample - loss: 1.1966 - acc: 0.6468 - val_loss: 0.3049 - val_acc: 0.8916
Epoch 2/15
504/504 [==============================] - 66s 132ms/sample - loss: 0.3286 - acc: 0.8770 - val_loss: 0.1295 - val_acc: 0.9458
Epoch 3/15
504/504 [==============================] - 67s 133ms/sample - loss: 0.1547 - acc: 0.9345 - val_loss: 0.0703 - val_acc: 0.9819
Epoch 4/15
504/504 [==============================] - 64s 126ms/sample - loss: 0.0798 - acc: 0.9583 - val_loss: 0.0679 - val_acc: 0.9759
Epoch 5/15
504/504 [==============================] - 67s 133ms/sample - loss: 0.0634 - acc: 0.9722 - val_loss: 0.0867 - val_acc: 0.9819
Epoch 6/15
504/504 [==============================] - 67s 133ms/sample - loss: 0.0622 - acc: 0.9742 - val_loss: 0.0461 - val_acc: 0.9940
Epoch 7/15
504/504 [==============================] - 68s 134ms/sample - loss: 0.0576 - acc: 0.9782 - val_loss: 0.0717 - val_acc: 0.